In [ ]:
import os.path
import pickle

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.modify', 
          'https://www.googleapis.com/auth/spreadsheets']

def main():
    """Shows basic usage of the Gmail API and Google Sheets API.
    Lists the user's Gmail labels and adds new emails to a Google Sheets document.
    """
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '../credentials/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    gmail_service = build('gmail', 'v1', credentials=creds)
    sheets_service = build('sheets', 'v4', credentials=creds)

    # Fetch all rows from the sheet first
    sheet = sheets_service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId="SPREADSHEETID",
        range="Sheet1!A:B"
    ).execute()
    values = result.get('values', [])

    results = gmail_service.users().messages().list(userId='me', q="is:unread from:SENDERSMAIL").execute()
    messages = results.get('messages', [])

    if not messages:
        print('No new messages.')
    else:
        print('New job ads:')
        for message in messages:
            msg = gmail_service.users().messages().get(userId='me', id=message['id']).execute()
            headers = msg['payload']['headers']
            subject = [i['value'] for i in headers if i['name']=='Subject']
            link = f"https://mail.google.com/mail/u/0/#inbox/{msg['id']}"

            print(f"Subject: {subject[0]}\nLink: {link}")

            # Check if this email is already in the sheet
            if [subject[0], link] in values:
                print("There is nothing new")
            else:
                # Add new email to Google Sheets document
                value_range = {
                    "range": "Sheet1!A1",
                    "majorDimension": "ROWS",
                    "values": [
                        [subject[0], link]
                    ],
                }
                sheet.values().append(
                    spreadsheetId="SPREADSHEETID",
                    range="Sheet1!A1",
                    valueInputOption="USER_ENTERED",
                    body=value_range,
                ).execute()

                # Mark the email as read
                gmail_service.users().messages().modify(
                    userId='me',
                    id=message['id'],
                    body={ 'removeLabelIds': ['UNREAD'] }
                ).execute()

if __name__ == '__main__':
    main()
